# 02. AWS SageMaker Notebooks and SDK

* 딥리워드 / 딥스케일 - AWS SageMaker 기초 [1]
* 발표자 : 김무성

----------------------------

# 차례
* Exercise 2.1: Create Your Notebook Instance
    - 1. Create an Amazon SageMaker notebook instance
    - 2. Download Pandas-Demo.ipynb to the notebook instance
    - 3. Run Pandas-Demo.ipynb on the notebook instance
* Exercise 2.2: Amazon SageMaker SDK
    - 1. Download notebook code to the notebook instance
    - 2. Run the notebook code on the notebook instance
    - 3. End running Jupyter processes, and stop the notebook instance
    - 4. Delete the endpoint con guration and model

--------------------------------------

<img src="figures/sagemaker_overview.png" width=600 />

# Exercise 2.1: Create Your Notebook Instance

##### 참고자료
* [1] edX course : Amazon SageMaker: Simplifying Machine Learning Application Development 
    - https://www.edx.org/course/simplifying-machine-learning-app-development-with-amazon-sagemaker
    - Week 2 . Amazon SageMaker Notebooks and SDK

### 0. edXSageMakerUser IAM user 로 로그인

###### 01 chapter 발표자료 참조
01_intro / 01_intro_aws_sagemaker.ipynb
- https://nbviewer.jupyter.org/github/deepreward/DeepScale/blob/master/aws_sagemaker_basic/01_intro/01_intro_aws_sagemaker.ipynb


-------------

* Sign in as the edXSageMakerUser IAM user (IAM user name: SageMakerOnAWS)
* https://your-sign-in-URL

### 1. Create an Amazon SageMaker notebook instance

* Sign in to the AWS Management Console as the edXSageMakerUser IAM user.
* In the console, click Services > Amazon SageMaker to open the Amazon SageMaker dashboard. 
* Make sure you are in the Oregon Region.
* In the left navigation pane, click Notebook instances > Create notebook instance.
* For Notebook instance name, enter edXSageMaker in the text box.
* Confirm that the IAM role is populated with the role you created in the previous exercise.
* Click Create notebook instance.

<img src="figures/cap01.jpeg" width=600 />
<img src="figures/cap02.jpeg" width=600 />
<img src="figures/cap03.jpeg" width=600 />

### 2. Download Pandas-Demo.ipynb to the notebook instance

* Click the Open link for your new edXSageMaker notebook instance. The Jupyter notebook home opens in a new browser tab.
* To download the Pandas-Demo.ipynb notebook, open a terminal window, and for New, click Terminal. A terminal window displays where you can run commands on your notebook instance.

<img src="figures/cap04.jpeg" width=600 />

<img src="figures/cap05.jpeg" width=600 />

* Run the following commands to change the directory and download the notebook:

```shell
$ cd ~/SageMaker/
$ wget https://us-west-2-tcdev.s3.amazonaws.com/courses/AWS-100-MLS/v1.0.0/exercises/notebooks/Pandas-Demo.ipynb
```

<img src="figures/cap06.jpeg" width=600 />

### 3. Run Pandas-Demo.ipynb on the notebook instance.

<img src="figures/cap07.jpeg" width=600 />

-------------------------------

# Exercise 2.2: Amazon SageMaker SDK

The exercises in this course will generate associated charges in your AWS account. In this exercise, you will create the following:
* Amazon S3 bucket, with training and model S3 objects 
* Amazon SageMaker ML training instance
* Amazon SageMaker ML hosting instance

This exercise introduces you to the Amazon SageMaker high-level Python library. The library is provided as an abstraction for training and deploying machine-learned models on Amazon SageMaker.

You will repeat the exercise 1.1 Driving Linear Learner steps for the AWS console, but instead of using the console, you will interact with Amazon SageMaker using the high-level Python library from a notebook instance.

### 1. Download notebook code to the notebook instance

In this section, you will download the Amazon-SageMaker-high-level-Python-library.ipynb notebook and training.csv  le to your instance.

<img src="figures/cap03.jpeg" width=600 />

* To download the Amazon-SageMaker-high-level-Python-library.ipynb notebook, open a terminal window. For New, click Terminal.
* Run the following commands to change the directory, and download the notebook and .csv file.

```shell
$ cd ~/SageMaker/
$ wget https://us-west-2-tcdev.s3.amazonaws.com/courses/AWS-100-MLS/v1.0.0/exercises/notebooks/Amazon-SageMaker-high-level-Python-library.ipynb
$ wget https://us-west-2-tcdev.s3.amazonaws.com/courses/AWS-100-MLS/v1.0.0/exercises/notebooks/training.csv
```

<img src="figures/cap08.jpeg" width=600 />
<img src="figures/cap09.jpeg" width=600 />

### 2. Run the notebook code on the notebook instance

* You now have everything on the notebook instance you need to run the high-level library demonstration. As with the previous notebook exercise, you can complete the exercise by running each cell in the notebook.
* Ensure you run the  nal cell in the notebook to delete the endpoint created in this exercise.

##### 데이터 로드

```python
import pandas as pd
import numpy as np

df = pd.read_csv('training.csv', header=None)
matrix = df.as_matrix()

%matplotlib inline
import matplotlib.pyplot as plt

def show_road(img, caption='', subplot=None):
    if subplot==None:
        _,(subplot)=plt.subplots(1,1)
    imgr=img.reshape((10,25))
    subplot.axis('off')
    subplot.imshow(imgr, cmap='gray')
    plt.title(caption)

row = 15
show_road(matrix[row][1:], "Label = {}".format(matrix[row][0]))
```

<img src="figures/cap10.jpeg" width=400 />

##### SageMaker 라이브러리 

```python
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
```

##### 머신러닝 모델 생성

```python
multiclass_estimator = sagemaker.LinearLearner(role=role,
                                               train_instance_count=1,
                                               train_instance_type='ml.m4.xlarge',
                                               predictor_type='multiclass_classifier',
                                               num_classes=3)
```

##### 학습 - 자동으로 '훈련작업(training jab)' 인스턴스, s3  인스턴스 등의 관련 컴포넌트들을 생성, 연결  해준다.

```python
mini_batch_size = 100
records = matrix[:,:-1].astype('float32')
labels = matrix[:,0].astype('float32')
train_records = multiclass_estimator.record_set(records, labels, channel='train')
multiclass_estimator.fit([train_records])
```

<img src="figures/cap11.jpeg" width=600 />
<img src="figures/cap12.jpeg" width=600 />
<img src="figures/cap_s3.png" width=600 />

<img src="figures/cap13.jpeg" width=600 />
<img src="figures/cap14.png" width=600 />

##### 모델 deploy - 모델 구성, 엔드포인트 구성 및 생성 등의 작업을 자동으로 해준다.

```python
multiclass_predictor = multiclass_estimator.deploy(initial_instance_count=1, instance_type='ml.t2.medium')
```

<img src="figures/cap15.png" width=600 />
<img src="figures/cap16.png" width=600 />
<img src="figures/cap17.png" width=600 />
<img src="figures/cap18.png" width=600 />
<img src="figures/cap19.jpeg" width=600 />
<img src="figures/cap20.jpeg" width=600 />

##### 추론

```python
labels = ["left", "straight", "right"]

experiment = np.array([[0,0,0,0,1,2,2,2,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,2,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,2,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,2,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,2,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,2,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,2,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,2,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,2,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,2,2,2,1,0,0,0,0,0,0,0,0,0]]).astype('float32')
result = multiclass_predictor.predict(experiment)
predicted_label = int(result[0].label['predicted_label'].float32_tensor.values[0])
show_road(experiment, "Label = {}".format(labels[predicted_label]))

experiment = np.array([[0,0,0,0,0,0,0,0,1,2,2,2,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,2,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,2,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,2,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,2,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,2,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,2,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,2,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,2,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,2,2,2,1,0,0,0,0,0,0,0,0,0]]).astype('float32')
result = multiclass_predictor.predict(experiment)
predicted_label = int(result[0].label['predicted_label'].float32_tensor.values[0])
show_road(experiment, "Label = {}".format(labels[predicted_label]))
```

##### endpoint 삭제 - 실행하면 엔드포인트 이외의 것들(모델 등)은 남아있다.

```python
# delete endpoint
multiclass_predictor.delete_endpoint()
```

<img src="figures/cap21.png" width=600 />
<img src="figures/cap22.png" width=600 />
<img src="figures/cap23.png" width=600 />

---------------------------------

<font color="red"> 실습이 끝나면 요금폭탄을 맞지 않도록, 반드시 멈추고 지웁시다. </font>

### 3. End running Jupyter processes, and stop the notebook instance.

* From the Jupyter notebook home, click Running. 
* Click Shutdown next to the terminal and notebook.
* To stop the notebook instance, return to the AWS console, and click Services > Amazon SageMaker to open the Amazon SageMaker dashboard.
* In the left navigation pane, click Notebook instances, and then click Stop next to the edXSageMaker instance.

### 4. Delete the endpoint con guration and model.

* Return to the the Amazon SageMaker dashboard.
* In the left navigation pane, click Endpoints, and ensure the endpoint created by the notebook is removed.
* In the left navigation pane, click Endpoint con gurations, and choose the option for the endpoint that starts with linear-learner.
* For Actions, click Delete.
* Click Delete to confirm.
* In the left navigation pane, click Models, and choose the option for the model that starts with linear-learner. 
* For Actions, click Delete.
* Click Delete to confirm.

----------------------------------

# 참고자료
* [1] edX course : Amazon SageMaker: Simplifying Machine Learning Application Development 
    - https://www.edx.org/course/simplifying-machine-learning-app-development-with-amazon-sagemaker
    - Week 2 . Amazon SageMaker Notebooks and SDK
* [2] Amazon SageMaker high-level Python library 
    - https://github.com/aws/sagemaker-python-sdk 
    - https://sagemaker.readthedocs.io/en/latest/